In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input
from tensorflow.keras.preprocessing import image

In [3]:
# root_dir = "/content/drive/MyDrive/Colab Notebook Files/OCT2017"
root_dir = r"C:\Users\User\Downloads\archive\OCT2017"
out_dir = r"D:\dataset"

data = np.load(out_dir + "/thesis-dataset.npz")
for k, v in data.items():
    print(k, data[k].shape)

X_train (83484, 28, 28)
y_train (83484,)
X_test (0,)
y_test (0,)


In [4]:
X_train = data.get("X_train")

In [5]:
X_train.shape

(83484, 28, 28)

In [6]:
class Data(object):
    def load_data(self):
        real_data = X_train
        real_data = real_data[:2000].astype('float32')
        real_data = np.expand_dims(real_data, axis=3) / 255

        self.real_data = real_data
        
    def __init__(self):
        self.real_data = None
        self.load_data()


In [15]:
class C(object):
    @staticmethod
    def random_layer(last_output, filter_size):
        layer = np.random.choice([layers.Conv2D, layers.DepthwiseConv2D, layers.Conv2DTranspose],
                                 p=[0.20, 0.20, 0.60])

        hyper_parameters = {
            'filters': filter_size,
            'kernel_size': (lambda x: [x, x])(np.random.randint(1, 4)),
            'strides': (lambda x: [x, x])(np.random.randint(1, 4)),
            'padding': 'valid',
            'dilation_rate': (lambda x: [x, x])(np.random.randint(1, 4)),
            'activation': np.random.choice(['relu']),
        }

        if layer == layers.Conv2D:
            hyper_parameters[np.random.choice(['dilation_rate'])] = [1, 1]
            hyper_parameters['padding'] = 'same'

        elif layer == layers.DepthwiseConv2D:
            hyper_parameters[np.random.choice(['dilation_rate'])] = [1, 1]
            hyper_parameters['padding'] = 'same'
            hyper_parameters.pop('filters')

        elif layer == layers.Conv2DTranspose:
            if last_output.shape[1] > C.image_width:
                hyper_parameters['strides'] = [1, 1]
            else:
                hyper_parameters['strides'] = [2, 2]
                hyper_parameters['dilation_rate'] = [1, 1]

        while True:
            try:
                layer = layer(**hyper_parameters)(last_output)
                break
            except ValueError:
                last_output = layers.ZeroPadding2D(data_format='channels_last')(last_output)

        layer = layers.BatchNormalization()(layer)
        return layer

    @staticmethod
    def make_fixed_discriminator():
        model_input = layers.Input(shape=[C.image_width, C.image_width, 1]) 
        model_output = model_input
        for i in range(4):
            model_output = layers.Conv2D(filters=(2 ** i) * 32,
                                           kernel_size=[5, 5],
                                           strides=[2, 2],
                                           padding='same',
                                           )(model_output)
            model_output = layers.BatchNormalization()(model_output)
            model_output = layers.LeakyReLU()(model_output)

        model_output = layers.Lambda(lambda x: x * 2)(model_output)

        model_output = layers.Flatten()(model_output)
        model_output = layers.Dense(units=1, activation='linear')(model_output)

        model = keras.Model(inputs=model_input, outputs=model_output, name='discriminator')

        return model

    @staticmethod
    def make_random_discriminator():
        model_input = layers.Input(shape=[C.image_width, C.image_width, 1])
        model_output = model_input
        for i in range(C.discriminator_layer_size()):
            model_output = layers.Conv2D(filters=(2 ** i) * 32,
                                           kernel_size=[5, 5],
                                           strides=[2, 2],
                                           padding='same',
                                           )(model_output)
            model_output = layers.BatchNormalization()(model_output)
            model_output = layers.LeakyReLU()(model_output)

        model_output = layers.Lambda(lambda x: x * 2)(model_output)

        model_output = layers.Flatten()(model_output)
        model_output = layers.Dense(units=1, activation='linear')(model_output)

        model = keras.Model(inputs=model_input, outputs=model_output, name='discriminator')

        return model

    @staticmethod
    def make_fixed_generator():
        model_output = layers.Input(shape=[C.noise_dimension])
        model_input = model_output
        model_output = layers.Dense(units=7 * 7 * 128)(model_output)
        model_output = layers.Reshape([7, 7, 128])(model_output)
        model_output = layers.Conv2DTranspose(filters=64,
                                                kernel_size=[5, 5],
                                                strides=[2, 2],
                                                padding='same',
                                                activation='relu'
                                                )(model_output)
        model_output = layers.BatchNormalization()(model_output)
        model_output = layers.Conv2DTranspose(filters=32,
                                                kernel_size=[5, 5],
                                                strides=[2, 2],
                                                padding='same',
                                                activation='relu'
                                                )(model_output)
        model_output = layers.BatchNormalization()(model_output)
        model_output = layers.Conv2DTranspose(filters=1,
                                                kernel_size=[5, 5],
                                                strides=[1, 1],
                                                padding='same',
                                                activation='tanh'
                                                )(model_output)
        model_output = layers.Lambda(lambda x: (x + 1) / 2)(model_output)

        return keras.Model(inputs=model_input, outputs=model_output)

    @staticmethod
    def make_random_generator():
        model_input = layers.Input(shape=[C.noise_dimension])
        model_output =model_input
        model_output = layers.Dense(C.first_convolution_shape[0]
                                      * C.first_convolution_shape[1]
                                      * C.first_convolution_shape[2])(model_output)
        model_output = layers.Reshape(C.first_convolution_shape)(model_output)

        for i in reversed(range(C.generator_layer_size())):
            model_output = C.random_layer(model_output, 2 ** (i + 2))

        output_difference = keras.Model(inputs=model_input, outputs=model_output).output_shape[1] - C.image_width
        if output_difference >= 0:
            model_output = layers.Conv2D(
                filters=1,
                kernel_size=[output_difference + 1, output_difference + 1],
                activation='tanh',
            )(model_output)
        else:
            model_output = layers.Conv2DTranspose(
                filters=1,
                kernel_size=[-output_difference + 1, -output_difference + 1],
                activation='tanh',
            )(model_output)

        model_output = layers.Lambda(lambda x: (x + 1) / 2)(model_output)

        model = keras.Model(inputs=model_input, outputs=model_output, name='generator')

        return model

    @staticmethod
    def generator_layer_size():
        return np.random.randint(6, 8)

    @staticmethod
    def discriminator_layer_size():
        return np.random.randint(3, 5)

    image_width = 28
    noise_dimension = 128
    first_convolution_shape = np.array([4, 4, 128])
    gan_size = 3
    batch_size = 64
    learning_rate = 0.003

    path = r'D:\results'
    generate_image_size = 10
    
    
# fixed_generator_model = C.make_fixed_generator()
# fixed_generator_model.summary()

fixed_discriminator_model = C.make_fixed_discriminator()
fixed_discriminator_model.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 14, 14, 32)        832       
                                                                 
 batch_normalization_28 (Bat  (None, 14, 14, 32)       128       
 chNormalization)                                                
                                                                 
 leaky_re_lu_15 (LeakyReLU)  (None, 14, 14, 32)        0         
                                                                 
 conv2d_19 (Conv2D)          (None, 7, 7, 64)          51264     
                                                                 
 batch_normalization_29 (Bat  (None, 7, 7, 64)         256       
 chNormalization)                                    

In [16]:
fixed_generator_model = C.make_fixed_generator()
fixed_generator_model.summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 128)]             0         
                                                                 
 dense_9 (Dense)             (None, 6272)              809088    
                                                                 
 reshape_4 (Reshape)         (None, 7, 7, 128)         0         
                                                                 
 conv2d_transpose_14 (Conv2D  (None, 14, 14, 64)       204864    
 Transpose)                                                      
                                                                 
 batch_normalization_32 (Bat  (None, 14, 14, 64)       256       
 chNormalization)                                                
                                                                 
 conv2d_transpose_15 (Conv2D  (None, 28, 28, 32)       512

In [17]:
random_discriminator_model = C.make_random_discriminator()
random_discriminator_model.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_22 (Conv2D)          (None, 14, 14, 32)        832       
                                                                 
 batch_normalization_34 (Bat  (None, 14, 14, 32)       128       
 chNormalization)                                                
                                                                 
 leaky_re_lu_19 (LeakyReLU)  (None, 14, 14, 32)        0         
                                                                 
 conv2d_23 (Conv2D)          (None, 7, 7, 64)          51264     
                                                                 
 batch_normalization_35 (Bat  (None, 7, 7, 64)         256       
 chNormalization)                                    

In [18]:
random_generator_model = C.make_random_generator()
random_generator_model.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 128)]             0         
                                                                 
 dense_11 (Dense)            (None, 2048)              264192    
                                                                 
 reshape_5 (Reshape)         (None, 4, 4, 128)         0         
                                                                 
 conv2d_transpose_17 (Conv2D  (None, 8, 8, 256)        33024     
 Transpose)                                                      
                                                                 
 batch_normalization_37 (Bat  (None, 8, 8, 256)        1024      
 chNormalization)                                                
                                                                 
 depthwise_conv2d (Depthwise  (None, 8, 8, 256)        51

In [19]:
class Gan(object):
    def __init__(self, real_data, generator, discriminator, learning_rate):
        self.real_data = real_data
        optimizer = keras.optimizers.Adam(learning_rate)

        discriminator.trainable = True
        discriminator.compile(optimizer=optimizer, loss='mse')

        discriminator.trainable = False
        adversarial = keras.Model(inputs=generator.input, outputs=discriminator(generator.output))
        adversarial.compile(optimizer=optimizer, loss='mse')

        self.discriminator = discriminator
        self.generator = generator
        self.adversarial = adversarial

    def train_on_batch_index(self, batch_index):
        noise = np.random.uniform(-1, 1, [len(batch_index), C.noise_dimension]).astype('float32')
        fake_data = self.generator.predict(noise)
        real_data = self.real_data[batch_index]

        self.adversarial.train_on_batch(noise, np.ones([len(batch_index), 1]))
        self.discriminator.train_on_batch(real_data, np.ones([len(batch_index), 1]))
        self.discriminator.train_on_batch(fake_data, np.zeros([len(batch_index), 1]))

    def get_generator_weights(self):
        return self.generator.get_weights()

    def set_generator_weights(self, weights):
        self.generator.set_weights(weights)

    def get_discriminator_weights(self):
        return self.discriminator.get_weights()

    def set_discriminator_weights(self, weights):
        self.discriminator.set_weights(weights)

    def get_sample_images(self):
        noise = np.random.uniform(-1, 1, [C.generate_image_size, C.noise_dimension])
        image_arrays = self.generator.predict(x=noise) * 255.0

        return image_arrays

In [20]:
class DistributedGan(object):
    def __init__(self, distribution_size, real_data, generator, discriminator):
        self.discriminator = discriminator
        self.generator = generator
        self.gans = [Gan(real_data[np.random.choice(len(real_data), len(real_data))],
                         keras.models.clone_model(generator),
                         keras.models.clone_model(discriminator),
                         C.learning_rate * distribution_size) for _ in range(distribution_size)]

    def get_generator_weights(self):
        weights_set = [gan.get_generator_weights() for gan in self.gans]
        return np.mean(weights_set, axis=0)

    def set_generator_weights(self, weights):
        self.generator.set_weights(weights)
        [gan.set_generator_weights(weights) for gan in self.gans]

    def get_discriminator_weights(self):
        weights_set = ([gan.get_discriminator_weights() for gan in self.gans])
        return np.mean(weights_set, axis=0)

    def set_discriminator_weights(self, weights):
        self.discriminator.set_weights(weights)
        [gan.set_discriminator_weights(weights) for gan in self.gans]

    def train_on_batch_index(self, batch_index):
        [gan.train_on_batch_index(batch_index) for gan in self.gans]
        self.set_generator_weights(self.get_generator_weights())
        self.set_discriminator_weights(self.get_discriminator_weights())

    def save_generator(self, directory_path, iteration_number):
        generator_path = directory_path + '/generator'
        try:
            os.makedirs(generator_path)
        except FileExistsError:
            pass

        self.generator.save_weights(generator_path + '/weights.h5')
        with open(generator_path + '/architecture.json', 'w') as f:
            f.write(self.generator.to_json())
        keras.utils.plot_model(self.generator, generator_path + '/graph.png', show_shapes=True)

        image_path = directory_path + '/image'

        image_arrays = self.gans[0].get_sample_images()

        try:
            os.makedirs(image_path)
        except FileExistsError:
            pass

        for i in range(len(image_arrays)):
            image.save_img(x=image_arrays[i],
                           path=image_path + '/iteration%d num%d.png' % (iteration_number, i))

    def save_discriminator(self, directory_path):
        discriminator_path = directory_path + '/discriminator'
        try:
            os.makedirs(discriminator_path)
        except FileExistsError:
            pass

        self.discriminator.save_weights(discriminator_path + '/weights.h5')
        with open(discriminator_path + '/architecture.json', 'w') as f:
            f.write(self.discriminator.to_json())
        keras.utils.plot_model(self.discriminator, discriminator_path + '/graph.png', show_shapes=True)

In [21]:
class MultiDistributedGan(object):
    def __init__(self, real_data, distribution_size, generators, discriminators):
        self.same_generator_gans_group = [[] for _ in range(len(generators))]
        self.same_discriminator_gans_group = [[] for _ in range(len(discriminators))]
        self.gans = []

        for i in range(len(generators)):
            for j in range(len(discriminators)):
                distributed_gan = DistributedGan(distribution_size,
                                                 real_data[np.random.choice(len(real_data), len(real_data))],
                                                 keras.models.clone_model(generators[i]),
                                                 keras.models.clone_model(discriminators[j]))

                self.gans.append(distributed_gan)
                self.same_generator_gans_group[i].append(distributed_gan)
                self.same_discriminator_gans_group[j].append(distributed_gan)

    def get_generators_weights(self):
        generators_weights = []
        for same_generator_gans in self.same_generator_gans_group:
            weights_set = [gan.get_generator_weights() for gan in same_generator_gans]
            generators_weights.append(np.mean(weights_set, axis=0))

        return generators_weights

    def set_generators_weights(self, generators_weights):
        for same_generator_gans, generator_weights in zip(self.same_generator_gans_group, generators_weights):
            [gan.set_generator_weights(generator_weights) for gan in same_generator_gans]

    def get_discriminators_weights(self):
        discriminators_weights = []
        for same_discriminator_gans in self.same_discriminator_gans_group:
            weights_set = []
            for same_discriminator_gan in same_discriminator_gans:
                weights_set.append(same_discriminator_gan.get_discriminator_weights())
            discriminators_weights.append(np.mean(np.array(weights_set), axis=0))

        return discriminators_weights

    def set_discriminators_weights(self, discriminators_weights):
        for same_discriminator_gans, discriminator_weights\
                in zip(self.same_discriminator_gans_group, discriminators_weights):
            [gan.set_discriminator_weights(discriminator_weights) for gan in same_discriminator_gans]

    def train_on_batch_index(self, batch_index):
        [gan.train_on_batch_index(batch_index) for gan in self.gans]
        self.set_generators_weights(self.get_generators_weights())
        self.set_discriminators_weights(self.get_discriminators_weights())

    def save(self, iteration_number):
        for i in range(len(self.same_generator_gans_group)):
            distributed_gan = self.same_generator_gans_group[i][0]
            distributed_gan.save_generator(C.path + '/generator %d' % i, iteration_number)

        for i in range(len(self.same_discriminator_gans_group)):
            distributed_gan = self.same_discriminator_gans_group[i][0]
            distributed_gan.save_discriminator(C.path + '/discriminator %d' % i)


In [ ]:
def main():
    real_data = Data().real_data
    print('use fixed generator and discriminator for test? [y/n]')
    if input() == 'y':
        make_generator_function = C.make_fixed_generator
        make_discriminator_function = C.make_fixed_discriminator
    else:
        make_generator_function = C.make_random_generator
        make_discriminator_function = C.make_random_discriminator

    print('generator size :')
    generator_size = int(input())

    print('discriminator size :')
    discriminator_size = int(input())

    print('distribution size :')
    distribution_size = int(input())

    print('iteration size :')
    iteration_size = int(input())

    generators = [make_generator_function() for _ in range(generator_size)]
    discriminators = [make_discriminator_function() for _ in range(discriminator_size)]

    multi_distributed_gan = MultiDistributedGan(real_data, distribution_size, generators, discriminators)

    for i in range(iteration_size):
        if i % 10 == 0:
            multi_distributed_gan.save(i)
            print('iteration', i)

        batch_indexes = np.array_split(np.random.permutation(len(real_data)), int(len(real_data) / C.batch_size))
        for batch_index in batch_indexes:
            multi_distributed_gan.train_on_batch_index(batch_index)


main()


use fixed generator and discriminator for test? [y/n]
y
generator size :
1
discriminator size :
1
distribution size :
1
iteration size :
100
1/1 [==============================] - 0s 167ms/step
iteration 0
1/1 [==============================] - 0s 32ms/step
iteration 10
2/2 [==============================] - 0s 10ms/step
